In [5]:
###########################################     CAR COUNT >> 200  ########################################################

#IMPORTING LIBRARIES
import cv2
import numpy as np

# OPENING THE VIDEO FILE
video_capture = cv2.VideoCapture('trafficsort.mp4')

# INITIALIZING THE BACKGROUND SUBTRACTOR AND CAR COUNTER
fgbg = cv2.createBackgroundSubtractorMOG2()
car_counter = 0

# READING THE VIDEO FRAME BY FRAME
while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    # APPLYING BACKGROUND SUBTRATION
    fgmask = fgbg.apply(frame)

    # REMOVING NOISE AND SMOOTHING THE MASK
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel=np.ones((6, 6), np.uint8))

    # FINDING CONTOURS IN THE MASK
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    #DRAWING BOUNDING BOXES AROUND DETCTED CONTOURS
    for contour in contours:
        if cv2.contourArea(contour) > 2000:  # ADJUSTING THE CONTOUR AREA AS PER THE DESIRED OUTPUT
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            car_counter += 1

    # ADDING THE CAR COUNTER TO THE FRAME OF THE VIDEO
    cv2.putText(frame, f'Car Count: {car_counter}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # DISPLAYING THE OUTPUT VIDEO
    cv2.imshow('Car Detection', frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
######################################################## CAR COUNT >> 200 ################################################################

In [8]:
################################################# CONGESTED == FAILURE ########################################################

# IMPORTING NECESSARY LIBRARIES
import cv2
import numpy as np

# CREATING A LINE AS A BOUNDARY TO COUNT CARS
counting_line = 500  # LINE POSITION CAN BE ADJUSTED AS NEEDED

# INITIALIZING THE CAR COUNT AND A SET CAR_ID THAT STORES THE DETECTED CARS 
car_count = 0
car_ids = set()

# OPENING THE VIDEO FILE
video_capture = cv2.VideoCapture('trafficsort.mp4')

# INITIALIZING THE BACKGROUND SUBTRACTOR
fgbg = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=120, detectShadows=False)

while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    # APPLYING BACKGROUND SUBTRACTOR
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel=np.ones((8, 8), np.uint8))

    # FINDING THE CONTOURS IN THE MASK
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 8000: # ADJUSTING THE CONTOUR AREA AS PER REQUIRMNETS 
            x, y, w, h = cv2.boundingRect(contour)

            # CHECK IF THE OBJECTS CROUSING THE LINE HAVE BEEN COUNTED
            if y + h > counting_line and id(contour) not in car_ids:
                car_count += 1
                car_ids.add(id(contour)) #MARK THE OBJECT AS COUNTED

            # PLOTTING THE BOUNDING BOX AROUND DETECTED CARS
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)


    #DRAWING THE LINE IN THE FRAME
    cv2.line(frame, (0, counting_line), (frame.shape[1], counting_line), (0, 0, 255), 2)

    #INITIALISING A LIST TO STORE THE BOUNDING BOXES THAT ARE PLOTTED
    d=[]
    d.append(cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)) #APPENDING THOSE BOXES IN THE LIST

    #DEFINING THE CONGESTION DENSITY AND THE THRESHOLD FOR THE CONGESTION TO DEFINE ROAD AS CONGESTED OR NOT CONGESTED
    congestion_density = len(d)
    congestion_threshold = 40

    #SETTING A CONDITION TO CLASSIFY THE ROAD IN THE FOOTAGE AS CONGESTED OR NOT CONGESTED
    if congestion_density > congestion_threshold:
        congestion_status = "Congested"
        frame = cv2.putText(frame, f"Congestion: {congestion_status}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        congestion_status = "Not Congested"
        frame = cv2.putText(frame, f"Congestion: {congestion_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    cv2.imshow('Car Detection', frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


In [ ]:
############################################FINAL CODE############################################################################

In [3]:
###################################################  FINAL IOT PROJECT(YOLO)  ###########################################################

#IMPORTING NECESSARY LIBRARIES
import cv2
import numpy as np

# LOADING THE YOLO MODEL
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# LOADING THE FILE WITH CLASSES
with open("coco128.yaml", "r") as f:

    classes = f.read().strip().split("\n")

# OPENING THE VIDEO FILE AND READING THE FILE FRAME BY FRAME
video_capture = cv2.VideoCapture('traffic.mp4')

while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    # PERFORMING OBJECT DETECTION
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False) #CONVERTS A FRAME INTO A BLOB FOR COMPATIBILTY WITH DNN
    net.setInput(blob) #PREPROCESSES THESE FRAMES OR BLOBS
    outs = net.forward(net.getUnconnectedOutLayersNames()) #PREPROCESSES THE BLOBS AND STORED THE PROCESSED FRAMES IN THE OUTS VARIABLE

    # ANALYSING THE DETECTIONS AND DRAWING BOUNDING BOXES
    conf_threshold = 0.3 #CONFIDENCE THRESHOLD (THE CONTOURS BELOW THIS THRESHOLD WILL BE IGNORED)
    nms_threshold = 0.5
    boxes = []
    for out in outs: #ITERATES OVER EACH OUTPUT FRAME
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x, center_y, width, height = (detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]))
                x, y = int(center_x - width / 2), int(center_y - height / 2)
                boxes.append([x, y, int(width), int(height)])

    # DRAWING BOUNDING BOXES ON THE VIDEO FRAME
    for box in boxes:
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    #DEFINING THE CINGESTION DENSITY AND IT'S THRESHOLD TO CLASSIFY THE ROAD IN THE FOOTAGE AS CONGESTED OR NOT CONGESTED
    crowd_density = len(boxes) #DENSITY IS EQUAL TO THE NUMBER OF BOXES PLOTTED OR THE NUMBER PF CARS DETECTED ON THE ROAD
    congestion_threshold = 40  #THRESHOLD SET TO 40 BOUNDING BOXES .I.E. 40 DETECTED CARS ON A ROAD

    #SETTING A CONDITION TO CLASSIFY THE ROAD IN THE VIDEO AS CONGESTED OR NOT CONGESTED
    if crowd_density > congestion_threshold:
        congestion_status = "Congested"
        frame = cv2.putText(frame, f"Congestion: {congestion_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        congestion_status = "Not Congested"
        frame = cv2.putText(frame, f"Congestion: {congestion_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    #DISPLAYING THE FINAL OUTPUT FRAME
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


#SENDING A BROADCAST MESSAGE USING WHATSAPP TO USERS IF THE ROAD IS CONGESTED USING PYWHATKIT LIBRARY
import pywhatkit
lst=['+918779790683','+917223006950']
for i in lst:
     if congestion_status == 'Congested':
        pywhatkit.sendwhatmsg_instantly(i,"The road:XYZ is congested, if you are planning to travel through this route, you are advised to take an alternate path to avoid congestion",4,True,4)

